# Model Fine-Tuning

## Import Libraries

In [1]:
import numpy as np
import os, torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, model_train, model_eval

## CIFAR-10 SSL Fix

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Set Random Seed & Device

In [3]:
rand_seed = 100
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

## Download & Pre-Process Datasets

### MNIST Dataset

In [4]:
mnist_mean = 0.5
mnist_std = 0.5
mnist_dim = 28

mnist_min, mnist_max = get_clip_bounds(mnist_mean,
                                       mnist_std,
                                       mnist_dim)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=0.0,
        std=np.divide(1.0, mnist_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, mnist_std),
        std=1.0)])

mnist_temp = datasets.MNIST(root='datasets/mnist', train=True,
                            download=True, transform=mnist_tf_train)
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

mnist_test = datasets.MNIST(root='datasets/mnist', train=False,
                            download=True, transform=mnist_tf)

### CIFAR-10 Dataset

In [5]:
cifar_mean = [0.491, 0.482, 0.447]
cifar_std = [0.202, 0.199, 0.201]
cifar_dim = 32

cifar_min, cifar_max = get_clip_bounds(cifar_mean,
                                       cifar_std,
                                       cifar_dim)
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(
        size=cifar_dim,
        padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=np.divide(1.0, cifar_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, cifar_mean),
        std=[1.0, 1.0, 1.0])])

cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True,
                              download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False,
                              download=True, transform=cifar_tf)

cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


## Dataset Loaders

In [6]:
batch_size = 128
workers = 4

mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

## Training Parameters

In [7]:
train_model = True

epochs = 50

lr = 0.004
lr_nin = 0.01
lr_scale = 0.5

momentum = 0.9

print_step = 5

if not os.path.isdir('weights/finetuned'):
    os.makedirs('weights/finetuned', exist_ok=True)

## MNIST Fine-Tuning

### LeNet Model

In [8]:
model = LeNet_MNIST().to(device)

if train_model:
    model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth'))
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/finetuned/mnist_lenet.pth')

model.load_state_dict(torch.load('weights/finetuned/mnist_lenet.pth'))
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.9981,  Train Loss : 0.0067
      Val Acc : 0.9886,    Val Loss : 0.0521
Epoch [5]
    Train Acc : 0.9991,  Train Loss : 0.0043
      Val Acc : 0.9870,    Val Loss : 0.0559
Epoch [10]
    Train Acc : 0.9994,  Train Loss : 0.0036
      Val Acc : 0.9866,    Val Loss : 0.0580
Epoch [15]
    Train Acc : 0.9995,  Train Loss : 0.0029
      Val Acc : 0.9871,    Val Loss : 0.0611
Epoch [20]
    Train Acc : 0.9995,  Train Loss : 0.0027
      Val Acc : 0.9862,    Val Loss : 0.0660
Epoch [25]
    Train Acc : 0.9998,  Train Loss : 0.0019
      Val Acc : 0.9869,    Val Loss : 0.0652
Epoch [30]
    Train Acc : 0.9998,  Train Loss : 0.0018
      Val Acc : 0.9864,    Val Loss : 0.0673
Epoch [35]
    Train Acc : 0.9998,  Train Loss : 0.0013
      Val Acc : 0.9865,    Val Loss : 0.0687
Epoch [40]
    Train Acc : 0.9998,  Train Loss : 0.0012
      Val Acc : 0.9864,    Val Loss : 0.0694
Epoch [45]
    Train Acc : 0.9999,  Train Loss : 0.0011
      Val Acc : 0.9867,    Val Loss :

### FC Model

In [9]:
model = FC_500_150().to(device)

if train_model:
    model.load_state_dict(torch.load('weights/clean/mnist_fc.pth'))
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/finetuned/mnist_fc.pth')

model.load_state_dict(torch.load('weights/finetuned/mnist_fc.pth'))
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.9954,  Train Loss : 0.0209
      Val Acc : 0.9730,    Val Loss : 0.0945
Epoch [5]
    Train Acc : 0.9965,  Train Loss : 0.0178
      Val Acc : 0.9735,    Val Loss : 0.0962
Epoch [10]
    Train Acc : 0.9973,  Train Loss : 0.0152
      Val Acc : 0.9714,    Val Loss : 0.0985
Epoch [15]
    Train Acc : 0.9974,  Train Loss : 0.0137
      Val Acc : 0.9718,    Val Loss : 0.0993
Epoch [20]
    Train Acc : 0.9982,  Train Loss : 0.0122
      Val Acc : 0.9728,    Val Loss : 0.0955
Epoch [25]
    Train Acc : 0.9987,  Train Loss : 0.0106
      Val Acc : 0.9728,    Val Loss : 0.1010
Epoch [30]
    Train Acc : 0.9989,  Train Loss : 0.0093
      Val Acc : 0.9727,    Val Loss : 0.1018
Epoch [35]
    Train Acc : 0.9993,  Train Loss : 0.0085
      Val Acc : 0.9725,    Val Loss : 0.1034
Epoch [40]
    Train Acc : 0.9993,  Train Loss : 0.0074
      Val Acc : 0.9735,    Val Loss : 0.1022
Epoch [45]
    Train Acc : 0.9995,  Train Loss : 0.0066
      Val Acc : 0.9726,    Val Loss :

## CIFAR-10 Fine-Tuning

### Network-In-Network Model

In [10]:
model = Net().to(device)

if train_model:
    model.load_state_dict(torch.load('weights/clean/cifar_nin.pth'))
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              cifar_loader_train, cifar_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/finetuned/cifar_nin.pth')

model.load_state_dict(torch.load('weights/finetuned/cifar_nin.pth'))
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.9566,  Train Loss : 0.1241
      Val Acc : 0.9788,    Val Loss : 0.0646
Epoch [5]
    Train Acc : 0.9585,  Train Loss : 0.1176
      Val Acc : 0.9785,    Val Loss : 0.0653
Epoch [10]
    Train Acc : 0.9610,  Train Loss : 0.1110
      Val Acc : 0.9802,    Val Loss : 0.0641
Epoch [15]
    Train Acc : 0.9615,  Train Loss : 0.1072
      Val Acc : 0.9809,    Val Loss : 0.0597
Epoch [20]
    Train Acc : 0.9640,  Train Loss : 0.1024
      Val Acc : 0.9785,    Val Loss : 0.0614
Epoch [25]
    Train Acc : 0.9637,  Train Loss : 0.1047
      Val Acc : 0.9775,    Val Loss : 0.0644
Epoch [30]
    Train Acc : 0.9632,  Train Loss : 0.1026
      Val Acc : 0.9802,    Val Loss : 0.0590
Epoch [35]
    Train Acc : 0.9643,  Train Loss : 0.0990
      Val Acc : 0.9786,    Val Loss : 0.0629
Epoch [40]
    Train Acc : 0.9671,  Train Loss : 0.0930
      Val Acc : 0.9787,    Val Loss : 0.0610
Epoch [45]
    Train Acc : 0.9669,  Train Loss : 0.0944
      Val Acc : 0.9761,    Val Loss :

### LeNet Model

In [11]:
model = LeNet_CIFAR().to(device)

if train_model:
    model.load_state_dict(torch.load('weights/clean/cifar_lenet.pth'))
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              cifar_loader_train, cifar_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/finetuned/cifar_lenet.pth')

model.load_state_dict(torch.load('weights/finetuned/cifar_lenet.pth'))
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.8628,  Train Loss : 0.3887
      Val Acc : 0.7930,    Val Loss : 0.6351
Epoch [5]
    Train Acc : 0.8766,  Train Loss : 0.3523
      Val Acc : 0.7947,    Val Loss : 0.6169
Epoch [10]
    Train Acc : 0.8815,  Train Loss : 0.3346
      Val Acc : 0.7930,    Val Loss : 0.6335
Epoch [15]
    Train Acc : 0.8873,  Train Loss : 0.3164
      Val Acc : 0.7932,    Val Loss : 0.6558
Epoch [20]
    Train Acc : 0.8930,  Train Loss : 0.2922
      Val Acc : 0.7934,    Val Loss : 0.6877
Epoch [25]
    Train Acc : 0.9012,  Train Loss : 0.2791
      Val Acc : 0.7915,    Val Loss : 0.6800
Epoch [30]
    Train Acc : 0.9046,  Train Loss : 0.2677
      Val Acc : 0.8006,    Val Loss : 0.6546
Epoch [35]
    Train Acc : 0.9091,  Train Loss : 0.2535
      Val Acc : 0.7909,    Val Loss : 0.6998
Epoch [40]
    Train Acc : 0.9152,  Train Loss : 0.2418
      Val Acc : 0.7966,    Val Loss : 0.6876
Epoch [45]
    Train Acc : 0.9191,  Train Loss : 0.2269
      Val Acc : 0.7995,    Val Loss :